In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

dr_name=[]
fl_names=[]
fl_name=[]
path=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    dr_name.append(dirname)
    fl_names.append(filenames)
    
    for filename in filenames:
        fl_name.append(filename)
        a=os.path.join(dirname, filename)
        path.append(a)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import itertools

from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils.vis_utils import plot_model

In [5]:
df = pd.read_csv('../input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [6]:
df[df['image_id']=='ISIC_0031633']

,lesion_id,image_id,dx,dx_type,age,sex,localization
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [7]:
df.head(10)

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back


In [8]:
df['dx'].value_counts()

nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

In [9]:
df['dx']

0          bkl
1          bkl
2          bkl
3          bkl
4          bkl
         ...  
10010    akiec
10011    akiec
10012    akiec
10013    akiec
10014      mel
Name: dx, Length: 10015, dtype: object

In [10]:
labels_dictionary_dx_num = {'nv':0, 'mel':1, 'bkl':2, 'bcc':3, 'akiec':4, 'vasc':5, 'df':6}


In [11]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [12]:
base_skin_dir = '../input/skin-cancer-mnist-ham10000'
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}
print(type(imageid_path_dict))

<class 'dict'>


In [13]:
df['path'] = df['image_id'].map(imageid_path_dict.get)
df['cell_type'] = df['dx'].map(lesion_type_dict.get)
df['cell_type_idx'] = pd.Categorical(df['cell_type']).codes

print(df['path'][10],"   ",df['image_id'][10])
print(len(df['path']))




../input/skin-cancer-mnist-ham10000/ham10000_images_part_1/ISIC_0025276.jpg     ISIC_0025276
10015


In [14]:
from PIL import Image as pl
im = pl.open("../input/skin-cancer-mnist-ham10000/ham10000_images_part_1/ISIC_0025276.jpg")
im=np.array(im)
print(im.shape)

(450, 600, 3)


In [15]:
labels = [labels_dictionary_dx_num[i] for i in df['dx']]
print(len(labels))

10015


In [16]:
df.isna().sum()
df['age'].fillna((df['age'].mean()), inplace=True)
df.isna().sum().sum()


0

In [17]:
from PIL import Image
print(len(df['path']))
data = []

for image_path in df['path']:
    image = Image.open(image_path)
    image_resised_71_71=image.resize((71,71))
    image_resised_71_71 = np.array(image_resised_71_71)
#     image_resised_224_224=image.resize((224,224))
#     image_resised_256_256=image.resize((256,256))
#     image_resised_100_100 = np.array(image_resised_100_100)
#     image_resised_224_224 = np.array(image_resised_224_224)
#     image_resised_256_256 = np.array(image_resised_256_256)
    data.append(image_resised_71_71)

10015


In [18]:
print(df['path'])

0        ../input/skin-cancer-mnist-ham10000/ham10000_i...
1        ../input/skin-cancer-mnist-ham10000/ham10000_i...
2        ../input/skin-cancer-mnist-ham10000/ham10000_i...
3        ../input/skin-cancer-mnist-ham10000/ham10000_i...
4        ../input/skin-cancer-mnist-ham10000/ham10000_i...
                               ...                        
10010    ../input/skin-cancer-mnist-ham10000/ham10000_i...
10011    ../input/skin-cancer-mnist-ham10000/ham10000_i...
10012    ../input/skin-cancer-mnist-ham10000/ham10000_i...
10013    ../input/skin-cancer-mnist-ham10000/ham10000_i...
10014    ../input/skin-cancer-mnist-ham10000/ham10000_i...
Name: path, Length: 10015, dtype: object


In [17]:
   
    image = Image.open(df['path'][4])
    image=image.resize((100,100))
    imageoo=np.array(image)

In [19]:
print(len(data))
print(len(labels))
print(data[0].shape)
print(data[5].shape)

10015
10015
(71, 71, 3)
(71, 71, 3)


In [65]:
import csv
import numpy as np
from PIL import Image

with open("data.csv", mode="w") as file:
    writer = csv.writer(file, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for filename, label in zip(data, labels):

        row_vector = filename.reshape(1, -1)

        # write the label and row vector to the file
        writer.writerow(row_vector.tolist()[0]+[label])


In [66]:
red=pd.read_csv("/kaggle/working/data.csv")
red.head()

,190,153,195,192,155,197,191,153.1,196,192.1,...,187.289,157.311,185.333,186.292,158.309,186.293,179.301,149.210,167.355,2
0,24,13,22,24,14,24,25,14,28,30,...,27,15,29,26,14,26,23,13,23,2
1,186,128,137,190,134,148,194,137,151,198,...,146,114,124,128,94,96,107,69,71,2
2,24,11,17,25,12,22,36,21,30,53,...,26,12,16,24,11,16,23,8,13,2
3,132,89,112,146,101,124,158,114,138,171,...,109,77,92,83,53,69,54,28,44,2
4,4,0,1,6,0,2,9,1,4,11,...,4,0,0,4,0,0,4,0,0,2


In [68]:
# define the column names as a list
column_names = ["pixel{}".format(i+1) for i in range(red.shape[1]-1)]+["label"] 

# assign the column names to the DataFrame
red.columns = column_names

# write the DataFrame back to the CSV file
red.to_csv("data_columns_labels.csv", index=False)

In [71]:
res=pd.read_csv("data_columns_labels.csv")
red.head(5)

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel29992,pixel29993,pixel29994,pixel29995,pixel29996,pixel29997,pixel29998,pixel29999,pixel30000,label
0,24,13,22,24,14,24,25,14,28,30,...,27,15,29,26,14,26,23,13,23,2
1,186,128,137,190,134,148,194,137,151,198,...,146,114,124,128,94,96,107,69,71,2
2,24,11,17,25,12,22,36,21,30,53,...,26,12,16,24,11,16,23,8,13,2
3,132,89,112,146,101,124,158,114,138,171,...,109,77,92,83,53,69,54,28,44,2
4,4,0,1,6,0,2,9,1,4,11,...,4,0,0,4,0,0,4,0,0,2


In [20]:
data=np.array(data)
labels=np.array(labels)
print(data.shape)
print(labels.shape)

(10015, 71, 71, 3)
(10015,)


In [21]:
X_train,X_test,Y_train,Y_test = train_test_split(data, labels, test_size=0.2, random_state=49)

In [22]:
print(X_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(Y_test.shape)

(8012, 71, 71, 3)
(2003, 71, 71, 3)
(2003,)
(2003,)


In [24]:
# scaling the data

X_train_scaled = X_train/255

X_test_scaled = X_test/255

In [25]:
import tensorflow as tf
from tensorflow import keras

In [26]:
num_of_classes = 7

# setting up the layers of Neural Network

model = keras.Sequential([
    
    keras.layers.Flatten(input_shape=(71,71,3)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(num_of_classes, activation='softmax')
])

In [27]:
# compile the neural network
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [28]:
# training the neural network
model.fit(X_train_scaled, Y_train, validation_split=0.1, epochs=10)

Epoch 1/10
226/226 [==============================] - 5s 5ms/step - loss: 1.4694 - acc: 0.6506 - val_loss: 1.0756 - val_acc: 0.6671
Epoch 2/10
226/226 [==============================] - 1s 4ms/step - loss: 1.0090 - acc: 0.6692 - val_loss: 1.1351 - val_acc: 0.6646
Epoch 3/10
226/226 [==============================] - 1s 4ms/step - loss: 0.9881 - acc: 0.6720 - val_loss: 1.0421 - val_acc: 0.6683
Epoch 4/10
226/226 [==============================] - 1s 4ms/step - loss: 0.9678 - acc: 0.6731 - val_loss: 1.1016 - val_acc: 0.6696
Epoch 5/10
226/226 [==============================] - 1s 4ms/step - loss: 0.9388 - acc: 0.6745 - val_loss: 1.0800 - val_acc: 0.6259
Epoch 6/10
226/226 [==============================] - 1s 5ms/step - loss: 0.9466 - acc: 0.6759 - val_loss: 1.1725 - val_acc: 0.6696
Epoch 7/10
226/226 [==============================] - 1s 4ms/step - loss: 0.9552 - acc: 0.6735 - val_loss: 0.9561 - val_acc: 0.6683
Epoch 8/10
226/226 [==============================] - 1s 4ms/step - loss: 0.

In [29]:
from tensorflow.keras import Sequential, models, layers
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import optimizers

In [41]:
convolutional_base = ResNet50(weights='imagenet', include_top=False, input_shape=(284,284,3))
convolutional_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 284, 284, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 290, 290, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 142, 142, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [45]:
num_of_classes = 7

model = models.Sequential()
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(convolutional_base)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Dense(num_of_classes, activation='softmax'))

In [46]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [47]:
history = model.fit(X_train_scaled, Y_train, validation_split=0.1, epochs=1,batch_size = 20)

361/361 [==============================] - 115s 208ms/step - loss: 1.3596 - acc: 0.5720 - val_loss: 1.3405 - val_acc: 0.6683


In [49]:
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['acc'])
history = model.fit(X_train_scaled, Y_train, validation_split=0.1, epochs=10,batch_size = 20)

Epoch 1/10
361/361 [==============================] - 109s 206ms/step - loss: 1.0910 - acc: 0.6666 - val_loss: 1.2488 - val_acc: 0.6683
Epoch 2/10
361/361 [==============================] - 73s 203ms/step - loss: 1.0367 - acc: 0.6681 - val_loss: 144.9303 - val_acc: 0.6072
Epoch 3/10
361/361 [==============================] - 73s 203ms/step - loss: 1.0419 - acc: 0.6738 - val_loss: 2.5688 - val_acc: 0.6459
Epoch 4/10
361/361 [==============================] - 73s 203ms/step - loss: 0.9573 - acc: 0.6714 - val_loss: 0.9371 - val_acc: 0.6633
Epoch 5/10
361/361 [==============================] - 73s 202ms/step - loss: 0.9307 - acc: 0.6774 - val_loss: 11834.2900 - val_acc: 0.4726
Epoch 6/10
361/361 [==============================] - 73s 203ms/step - loss: 0.9353 - acc: 0.6748 - val_loss: 599517.5625 - val_acc: 0.1072
Epoch 7/10
361/361 [==============================] - 73s 202ms/step - loss: 0.9253 - acc: 0.6713 - val_loss: 49401.1680 - val_acc: 0.4476
Epoch 8/10
361/361 [===================

In [50]:
loss, accuracy = model.evaluate(X_test_scaled, Y_test)
print('Test Accuracy =', accuracy)

63/63 [==============================] - 6s 86ms/step - loss: 403.3801 - acc: 0.6500
Test Accuracy = 0.650024950504303


In [51]:
SGD = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=SGD, loss='sparse_categorical_crossentropy', metrics=['acc'])
history = model.fit(X_train_scaled, Y_train, validation_split=0.1, epochs=1,batch_size = 20)

361/361 [==============================] - 88s 204ms/step - loss: 0.8807 - acc: 0.6839 - val_loss: 2998.7778 - val_acc: 0.6883


In [52]:
loss, accuracy = model.evaluate(X_test_scaled, Y_test)
print('Test Accuracy using SGD =', accuracy)

63/63 [==============================] - 5s 83ms/step - loss: 3407.6001 - acc: 0.6665
Test Accuracy using SGD = 0.6665002703666687
